<a href="https://colab.research.google.com/github/nsane9129/nlp/blob/Sentiment-Analysis/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
! pip install kaggle


In [11]:
! cp kaggle.json ~/.kaggle/


In [12]:
! chmod 600 ~/.kaggle/kaggle.json


In [5]:
! mkdir ~/.kaggle


In [17]:
!kaggle datasets download -d arkhoshghalb/twitter-sentiment-analysis-hatred-speech

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 61.7MB/s]


In [18]:
! unzip twitter-sentiment-analysis-hatred-speech


Archive:  twitter-sentiment-analysis-hatred-speech.zip
  inflating: test.csv                
  inflating: train.csv               


In [19]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [20]:
data=pd.read_csv('train.csv')

In [21]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
data.columns

In [24]:
len(data)

31962

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [25]:
data.dtypes

id        int64
label     int64
tweet    object
dtype: object

In [26]:
np.sum(data.isnull())

id       0
label    0
tweet    0
dtype: int64

# Data Preparation 

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
from nltk.corpus import stopwords
stopwords.words('english')


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

In [31]:
data_pos=data[data['label']==1]
data_negos=data[data['label']==0]

In [32]:
data_pos=data_pos.iloc[:int(20000)]
data_negos=data=data_negos.iloc[:int(20000)]

In [33]:
data=pd.concat([data_pos,data_negos])
data['tweet']=data['tweet'].str.lower()

In [34]:
stopwords_ =set(stopwords.words('english'))
def cleaning_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in stopwords_ ])
data['tweet'] = data['tweet'].apply(lambda w: cleaning_stopwords(w))
data['tweet'].head()





13    @user #cnn calls #michigan middle school 'buil...
14    comment! #australia #opkillingbay #seashepherd...
17                                       retweet agree!
23                @user @user lumpy says . prove lumpy.
34    unbelievable 21st century we'd need something ...
Name: tweet, dtype: object

# removing puctuation

In [36]:
punctuation_= string.punctuation
punctuation_all= punctuation_
def cleaning_punctuation(test):
  translator=str.maketrans('','',punctuation_all)
  return test.translate(translator)
  

In [37]:
data['tweet']=data['tweet'].apply(lambda t:cleaning_punctuation(t))
data['tweet'].tail()

21482             finding life inside illusion lack choice
21483    cold heaed people world anything get top peopl...
21484    user basically incompetent mental retardo losi...
21486    fuck land im boat motherfucker âµï¸ð lone...
21487    altwaystoheal healthy peace   think alternativ...
Name: tweet, dtype: object

re.sub(pattern, repl, string, count=0, flags=0) :
Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement repl. If the pattern isn’t found, string is returned unchanged. repl can be a string or a function; if it is a string, any backslash escapes in it are processed.

In [38]:
def cleaning_email(data):
  return re.sub('@[^\s]+', ' ', data)




In [39]:
data['tweet']=data['tweet'].apply(lambda t:cleaning_email(t))


In [40]:
def cleaning_url(data):
  return re.sub('((httph?://[^\s]+|www\.[^\s]+))', ' ', data)


In [41]:
data['tweet']=data['tweet'].apply(lambda u:cleaning_url(u))


In [42]:
def cleaning_numeric(data):
  return re.sub('[0-9]+', ' ', data)

In [43]:
data['tweet']=data['tweet'].apply(lambda n:cleaning_numeric(n))


tokeni*ation of tweet 

In [44]:
token=RegexpTokenizer(r'\w+')
data['tweet']=data['tweet'].apply(token.tokenize)

In [45]:
data['tweet'].tail()

21482      [finding, life, inside, illusion, lack, choice]
21483    [cold, heaed, people, world, anything, get, to...
21484    [user, basically, incompetent, mental, retardo...
21486    [fuck, land, im, boat, motherfucker, â, µï, ð,...
21487    [altwaystoheal, healthy, peace, think, alterna...
Name: tweet, dtype: object

In [46]:
stemmer=nltk.PorterStemmer()
def stemming_data(data):
  tweet=[stemmer.stem(word) for word in data]
  return data
  



In [47]:
data['tweet']=data['tweet'].apply(lambda t:stemming_data(t))

In [48]:
data['tweet'].tail()

21482      [finding, life, inside, illusion, lack, choice]
21483    [cold, heaed, people, world, anything, get, to...
21484    [user, basically, incompetent, mental, retardo...
21486    [fuck, land, im, boat, motherfucker, â, µï, ð,...
21487    [altwaystoheal, healthy, peace, think, alterna...
Name: tweet, dtype: object

https://medium.com/analytics-vidhya/introduction-bd62190f6acd
https://www.analyticsvidhya.com/blog/2021/06/natural-language-processing-sentiment-analysis-using-lstm/

In [54]:
X=data.tweet
y=data.label

max_input= 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_input)

X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.3, random_state=2)

Building the LSTM model using the ‘Keras’ library

In [65]:
def model():
    inputs = Input(name='inputs',shape=[max_input])#step1
    layer = Embedding(2000,50,input_length=max_length)(inputs) #step2
    layer = LSTM(64)(layer) #step3
    layer = Dense(256,name='FC1')(layer) #step4
    layer = Activation('relu')(layer) # step5
    layer = Dropout(0.5)(layer) # step6
    layer = Dense(1,name='out_layer')(layer) #step4 again but this time its giving only one output as because we need to classify the tweet as positive or negative
    layer = Activation('sigmoid')(layer) #step5 but this time activation function is sigmoid for only one output.
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it

In [70]:
lstm_model= model() # here we are calling the function of created model
lstm_model.compile(optimizer=RMSprop(),
  loss = 'categorical_crossentropy',
  metrics=['accuracy'])

In [72]:
print(lstm_model.summary())


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 500)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 500, 50)           100000    
                                                                 
 lstm_3 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [76]:
lstm_model.fit(X_train, Y_train, epochs = 5, batch_size=32)

Epoch 1/5
487/487 [==============================] - 137s 281ms/step - loss: 0.0000e+00 - accuracy: 0.9008
Epoch 2/5
487/487 [==============================] - 136s 278ms/step - loss: 0.0000e+00 - accuracy: 0.9008
Epoch 3/5
487/487 [==============================] - 135s 276ms/step - loss: 0.0000e+00 - accuracy: 0.9008
Epoch 4/5
487/487 [==============================] - 135s 277ms/step - loss: 0.0000e+00 - accuracy: 0.9008
Epoch 5/5
487/487 [==============================] - 135s 278ms/step - loss: 0.0000e+00 - accuracy: 0.9008


In [79]:
accuracy_model1 = lstm_model.evaluate(X_test,Y_test) #we are starting to test the model here


209/209 [==============================] - 12s 57ms/step - loss: 0.0000e+00 - accuracy: 0.8954


In [81]:
print('accuracy:{:0.2f}'.format(accuracy_model1[1]))

accuracy:0.90
